# Assignment 2

### Import Modules

In [16]:
import pandas as pd
import numpy as np
import nltk
#nltk.download('omw-1.4')
import re
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
#nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
#nltk.download('wordnet')
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC

### Load Dataset

In [3]:
# Load the data
data = pd.read_csv("IMDB Dataset.csv")

### Preprocess Data

In [4]:
# Preprocess the data
def preprocess_text(text):
    # Convert the text to lowercase
    text = text.lower()
    
    # Remove emails
    email_pattern = re.compile(r"(^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$)")
    text = re.sub(email_pattern, '', text)
    
    # Remove HTML tags
    text = BeautifulSoup(text, 'html.parser').get_text().strip()
    
    # Remove stop words
    stop_words = set(stopwords.words("english"))
    text = ' '.join([word for word in text.split() if word not in stop_words])
    
    # Lemmatize the text
    lemmatizer = WordNetLemmatizer()
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split(" ")])
    
    return text

#apply the preprocess_text function to the review column
data['review'] = data['review'].apply(preprocess_text)


C:\Users\Sasi\AppData\Local\Temp\ipykernel_17448\3708121857.py:11: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, 'html.parser').get_text().strip()


### Tokenization

In [5]:
# Tokenize the preprocessed text
tokenized_text = [review.split() for review in data['review']]

### Train Word2Vec Model - CBOW

In [6]:
# Train a Word2Vec model on the tokenized text
model = Word2Vec(tokenized_text, vector_size=100, window=5, min_count=1, sg=0)

# Convert the tokenized text to vectors using the trained Word2Vec model
# The vector for each review is the average of all the vectors of the words in the review
X = np.array([np.mean([model.wv[word] for word in review], axis=0) for review in tokenized_text])

# Split the data into training and testing sets
X_train_cbow, X_test_cbow, y_train, y_test = train_test_split(X, data['sentiment'], test_size=0.2)

### Train Word2Vec Model - Skip-gram

In [7]:
# Train a Word2Vec model on the tokenized text
model = Word2Vec(tokenized_text, vector_size=100, window=5, min_count=1, sg=1)

# Convert the tokenized text to vectors using the trained Word2Vec model
# The vector for each review is the average of all the vectors of the words in the review
X = np.array([np.mean([model.wv[word] for word in review], axis=0) for review in tokenized_text])

# Split the data into training and testing sets
X_train_skip_gram, X_test_skip_gram, y_train, y_test = train_test_split(X, data['sentiment'], test_size=0.2)

### Train and run Logistic Regression Model for Sentiment Analysis

In [8]:
# Train a logistic regression model on the training data
def logreg(X_train, X_test):
    # train a logistic regression classifier on the training set
    classifier = LogisticRegression()
    classifier.fit(X_train, y_train)
    y_test_pred = classifier.predict(X_test)
    return y_test_pred

### Train and run Random Forrest Model for Sentiment Analysis

In [9]:
def ranforr(X_train, X_test):
    # Train a random forest classifier on the training set
    classifier = RandomForestClassifier(n_estimators=100, random_state=42)
    classifier.fit(X_train, y_train)
    y_test_pred = classifier.predict(X_test)
    return y_test_pred

### Train and run Support Vector Machine 

In [19]:
def supvecmach(X_train, X_test):
    # Train a support vector machine classifier on the training set
    classifier = SVC(random_state=42)
    classifier.fit(X_train, y_train)
    y_test_pred = classifier.predict(X_test)
    return y_test_pred

### Evaluate Performance

In [10]:
# Evaluate the logistic regression model on the testing data
def evaluate(y_test_cbow, y_test_skip_gram):
    print("Accuracy-CBOW:", accuracy_score(y_test, y_test_cbow))
    print("Accuracy-Skip-gram:", accuracy_score(y_test, y_test_skip_gram))
    print("Precision-CBOW:", precision_score(y_test, y_test_cbow, pos_label='positive'))
    print("Precision-Skip-gram:", precision_score(y_test, y_test_skip_gram, pos_label='positive'))
    print("Recall-CBOW:", recall_score(y_test, y_test_cbow, pos_label='positive'))
    print("Recall-Skip-gram:", recall_score(y_test, y_test_skip_gram, pos_label='positive'))
    print("F1 Score-CBOW:", f1_score(y_test, y_test_cbow, pos_label='positive'))
    print("F1 Score-Skip-gram:", f1_score(y_test, y_test_skip_gram, pos_label='positive'))
    # Create a confusion matrix
    print("Confusion Matrix-CBOW", confusion_matrix(y_test, y_test_cbow))
    print("Confusion Matrix-Skip-gram", confusion_matrix(y_test, y_test_skip_gram))
    

### Logreg Performance

In [11]:
y_test_cbow = logreg(X_train_cbow, X_test_cbow)
y_test_skip_gram = logreg(X_train_skip_gram, X_test_skip_gram)

c:\Users\Sasi\anaconda3\envs\ANLP\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [12]:
evaluate(y_test_cbow, y_test_skip_gram)

Accuracy-CBOW: 0.4987
Accuracy-Skip-gram: 0.8691
Precision-CBOW: 0.49859606899318093
Precision-Skip-gram: 0.863905325443787
Recall-CBOW: 0.4972994598919784
Recall-Skip-gram: 0.8761752350470094
F1 Score-CBOW: 0.49794692038057087
F1 Score-Skip-gram: 0.8699970205581488
Confusion Matrix-CBOW [[2501 2500]
 [2513 2486]]
Confusion Matrix-Skip-gram [[4311  690]
 [ 619 4380]]


#### Evaulation CBOW vs skip-gram using logistic regression:

As we can see, the performance of skip-gram is significantly better than the performance of CBOW. Almost 3700 more texts are correctly assigned to their respective sentiment category

### Random Forrest Performance

In [13]:
y_test_cbow = ranforr(X_train_cbow, X_test_cbow)
y_test_skip_gram = ranforr(X_train_skip_gram, X_test_skip_gram)


In [14]:
evaluate(y_test_cbow, y_test_skip_gram)

Accuracy-CBOW: 0.4929
Accuracy-Skip-gram: 0.8409
Precision-CBOW: 0.4924210526315789
Precision-Skip-gram: 0.8345111896348646
Recall-CBOW: 0.46789357871574316
Recall-Skip-gram: 0.8503700740148029
F1 Score-CBOW: 0.47984408657298183
F1 Score-Skip-gram: 0.8423659962350143
Confusion Matrix-CBOW [[2590 2411]
 [2660 2339]]
Confusion Matrix-Skip-gram [[4158  843]
 [ 748 4251]]


#### Evaulation CBOW vs skip-gram using logistic regression:

As we can see, the performance of skip-gram is significantly better than the performance of CBOW. Almost 3500 more texts are correctly assigned to their respective sentiment category

### Support Vector Machine Perfomance

In [20]:
y_test_cbow = supvecmach(X_train_cbow, X_test_cbow)
y_test_skip_gram = supvecmach(X_train_skip_gram, X_test_skip_gram)

In [21]:
evaluate(y_test_cbow, y_test_skip_gram)

Accuracy-CBOW: 0.4994
Accuracy-Skip-gram: 0.8698
Precision-CBOW: 0.49922958397534667
Precision-Skip-gram: 0.863520157325467
Recall-CBOW: 0.4536907381476295
Recall-Skip-gram: 0.878375675135027
F1 Score-CBOW: 0.47537203940473693
F1 Score-Skip-gram: 0.8708845696152321
Confusion Matrix-CBOW [[2726 2275]
 [2731 2268]]
Confusion Matrix-Skip-gram [[4307  694]
 [ 608 4391]]


#### Evaulation CBOW vs skip-gram using logistic regression:

As we can see, the performance of skip-gram is significantly better than the performance of CBOW. Almost 3700 more texts are correctly assigned to their respective sentiment category